## NOMBRE: JAVIER GALINDO BERNAL
---------------------
# Read and clean a single file
------------------



In [1]:
import os
import sys
import logging
import pandas as pd
import datetime
import unicodedata

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'esp-big-data-javierg22'
directory = 'bigdata'

## Read Metadata
-----
General info for each relevant column

In [4]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

# datos-abiertos-julio-2019.csv


In [5]:
raw_data = get_input_data(filename = 'datos-abiertos-julio-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14560 entries, 0 to 14559
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INCIDENTE                    14560 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  14560 non-null  object
 2   CODIGO DE LOCALIDAD                14560 non-null  int64 
 3   LOCALIDAD                          14560 non-null  object
 4   EDAD                               14560 non-null  object
 5   UNIDAD                             14560 non-null  object
 6   GENERO                             14560 non-null  object
 7   RED                                14560 non-null  object
 8   TIPO_INCIDENTE                     14560 non-null  object
 9   PRIORIDAD                          14560 non-null  object
 10  MES                                14560 non-null  object
 11  CLASIFICACION FINAL                13035 non-null  object
dtypes: i

It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [6]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [7]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data["FECHA_INCIDENTE"] = pd.to_datetime(raw_data["FECHA_INCIDENTE"],dayfirst=True)
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [8]:
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14560 entries, 0 to 14559
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   FECHA_INCIDENTE                    14560 non-null  datetime64[ns]
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  14560 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   14560 non-null  int64         
 3   LOCALIDAD                          14560 non-null  object        
 4   EDAD                               14560 non-null  object        
 5   UNIDAD                             14560 non-null  object        
 6   GENERO                             14560 non-null  object        
 7   RED                                14560 non-null  object        
 8   TIPO_INCIDENTE                     14560 non-null  object        
 9   PRIORIDAD                          14560 non-null  object        
 10  MES                               

### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [9]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               1593
 Suba                  1548
Engativa               1521
 Bosa                  1122
 Ciudad Bolivar         994
Usaquen                 786
San Cristobal           776
 Rafael Uribe Uribe     741
Fontibon                723
 Puente Aranda          690
Usme                    636
Chapinero               599
Barrios Unidos          516
Los Martires            516
Santa Fe                489
Teusaquillo             464
Tunjuelito              462
 Antonio Nariño         292
 La Candelaria           91
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [10]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               1593
Suba                  1548
Engativa              1521
Bosa                  1122
Ciudad Bolivar         994
Usaquen                786
San Cristobal          776
Rafael Uribe Uribe     741
Fontibon               723
Puente Aranda          690
Usme                   636
Chapinero              599
Barrios Unidos         516
Los Martires           516
Santa Fe               489
Teusaquillo            464
Tunjuelito             462
Antonio Nariño         292
La Candelaria           91
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [11]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [12]:
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [13]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     6621
MASCULINO    4190
FEMENINO     3749
Name: GENERO, dtype: int64

In [14]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    4261
Inconsciente/Paro Cardiorrespiratorio        1623
Dificultad Respiratoria                      1398
Enfermo                                      1131
Convulsiones                                 1011
Heridos                                       747
Trastorno mental                              709
Dolor torácico                                611
Lesiones personales                           547
Intento de suicidio                           534
Patología Gineco - obstétrica                 425
Síntomas gastrointestinales                   344
Accidente cerebro vascular                    323
Caída                                         318
Intoxicaciones                                141
Ideas de suicidio                             111
Violencia Sexual                               41
Muerte Natural                                 39
SIN_DATO                                       39
Incendio estructural                           29


In [15]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Traslado                       6162
Cancelado                      3637
NaN                            1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [16]:
raw_data[col].fillna('SIN_DATO').value_counts()

Traslado                       6162
Cancelado                      3637
SIN_DATO                       1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

In [17]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [18]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [19]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 14560
Number of rows after cleaning: 13842


### Save the final table
use to_csv to save the table

In [20]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-julio-2019.csv', encoding='latin1', sep=';', index=False)

# datos-abiertos-agosto-2019.csv

In [21]:
raw_data = get_input_data(filename = 'datos-abiertos-agosto-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18141 entries, 0 to 18140
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18141 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  18141 non-null  object
 2   CODIGO DE LOCALIDAD                18141 non-null  int64 
 3   LOCALIDAD                          18141 non-null  object
 4   EDAD                               18141 non-null  object
 5   UNIDAD                             18141 non-null  object
 6   GENERO                             18141 non-null  object
 7   RED                                18141 non-null  object
 8   TIPO DE INCIDIDENT                 18141 non-null  object
 9   PRIORIDAD                          18141 non-null  object
 10  MES                                18141 non-null  object
 11  CLASIFICACION FINAL                18138 non-null  object
dtypes: i

In [22]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        "TIPO DE INCIDIDENT"                : "TIPO_INCIDENTE"
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


raw_data['LOCALIDAD'].value_counts(dropna=False)

In [23]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data.head()


,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [24]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2717
Engativa              1805
Suba                  1678
Bosa                  1264
Ciudad Bolivar        1220
Puente Aranda          977
San Cristobal          906
Usaquen                894
Rafael Uribe Uribe     892
Fontibon               878
Chapinero              707
Santa Fe               682
Usme                   637
Barrios Unidos         608
Teusaquillo            599
Tunjuelito             595
Los Martires           593
Antonio Nariño         369
La Candelaria          119
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [25]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [26]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [27]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)  

SIN_DATO     8617
MASCULINO    5143
FEMENINO     4381
Name: GENERO, dtype: int64

In [28]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6827
Inconsciente/Paro Cardiorrespiratorio        1875
Dificultad Respiratoria                      1504
Enfermo                                      1233
Convulsiones                                 1135
Heridos                                       847
Lesiones personales                           718
Trastorno mental                              670
Dolor torácico                                628
Intento de suicidio                           555
Síntomas gastrointestinales                   418
Accidente cerebro vascular                    401
Patología Gineco - obstétrica                 353
Caída                                         295
Intoxicaciones                                181
Ideas de suicidio                             102
SIN_DATO                                       58
Incendio estructural                           51
Muerte Natural                                 35
Violencia Sexual                               33


In [29]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
NaN                               3
Name: CLASIFICACION_FINAL, dtype: int64

In [30]:
raw_data[col].fillna('SIN_DATO').value_counts()

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
SIN_DATO                          3
Name: CLASIFICACION_FINAL, dtype: int64

In [31]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [34]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_agosto_2019.html')

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 16807
Number of rows after cleaning: 16807


In [36]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-agosto-2019.csv', encoding='latin1', sep=';', index=False)

# datos-abiertos-enero-2020.csv 


In [37]:
raw_data = get_input_data(filename = 'datos-abiertos-enero-2020.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15304 entries, 0 to 15303
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NUMERO_INCIDENTE     15304 non-null  object 
 1   FECHA_DESPACHO_518   15304 non-null  object 
 2   CODIGO DE LOCALIDAD  15304 non-null  int64  
 3   LOCALIDAD            15304 non-null  object 
 4   EDAD                 15304 non-null  int64  
 5   UNIDAD               15304 non-null  object 
 6   GENERO               15304 non-null  object 
 7   RED                  15304 non-null  object 
 8   TIPO_INCIDENTE       15304 non-null  object 
 9   PRIORIDAD            15304 non-null  int64  
 10  Unnamed: 10          0 non-null      float64
 11  Unnamed: 11          0 non-null      float64
 12  Unnamed: 12          0 non-null      float64
 13  Unnamed: 13          0 non-null      float64
 14  Unnamed: 14          0 non-null      float64
 15  Unnamed: 15          0 non-null     

In [38]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        "FECHA_DESPACHO_518"                : "FECHA_INICIO_DESPLAZAMIENTO_MOVIL"
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy              2116
 Engativá             1473
 Suba                 1465
 Bosa                 1072
 Ciudad Bolívar       1022
 Puente Aranda         942
 Fontibón              812
Usaquén                786
 San Cristóbal         746
Rafael Uribe Uribe     737
Usme                   678
Tunjuelito             550
Santa Fe               541
 Teusaquillo           527
 Barrios Unidos        506
Chapinero              498
 Los Mártires          446
 Antonio Nariño        320
 La Candelaria          64
Sumapaz                  3
Name: LOCALIDAD, dtype: int64

In [41]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2116
Engativá              1473
Suba                  1465
Bosa                  1072
Ciudad Bolívar        1022
Puente Aranda          942
Fontibón               812
Usaquén                786
San Cristóbal          746
Rafael Uribe Uribe     737
Usme                   678
Tunjuelito             550
Santa Fe               541
Teusaquillo            527
Barrios Unidos         506
Chapinero              498
Los Mártires           446
Antonio Nariño         320
La Candelaria           64
Sumapaz                  3
Name: LOCALIDAD, dtype: int64

In [42]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [43]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)  

SIN_DATO     7003
MASCULINO    4521
FEMENINO     3780
Name: GENERO, dtype: int64

In [45]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Heridos                                  6849
Inconsciente/Paro Cardiorrespiratorio    1520
Dificultad Respiratoria                  1359
Enfermo                                  1045
Convulsiones                              994
Trastorno mental                          661
Dolor torácico                            634
Intento de suicidio                       503
Accidente cerebro vascular                374
Patología Gineco - obstétrica             331
Síntomas gastrointestinales               331
Caída                                     320
Intoxicaciones                            138
Ideas de suicidio                          99
SIN_DATO                                   80
Violencia Sexual                           34
Quemaduras                                 10
Sangrado Vaginal                            8
Maltrato                                    7
Electrocución / Rescate                     7
Name: TIPO_INCIDENTE, dtype: int64

In [46]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

2020-01-04 08:06:26    10
2020-01-29 12:51:15     8
2020-01-26 03:37:17     7
2020-01-25 18:08:28     7
2020-01-22 09:37:57     6
                       ..
2020-01-12 22:41:58     1
2020-01-12 22:44:20     1
2020-01-12 22:49:49     1
2020-01-12 22:54:18     1
2020-02-01 01:19:50     1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 14958, dtype: int64

In [47]:
raw_data[col].fillna('SIN_DATO').value_counts()

2020-01-04 08:06:26    10
2020-01-29 12:51:15     8
2020-01-26 03:37:17     7
2020-01-25 18:08:28     7
2020-01-22 09:37:57     6
                       ..
2020-01-12 22:41:58     1
2020-01-12 22:44:20     1
2020-01-12 22:49:49     1
2020-01-12 22:54:18     1
2020-02-01 01:19:50     1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 14958, dtype: int64

In [48]:
col = 'UNIDAD'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_enero_2020.html')

Summarize dataset:   0%|          | 0/50 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 15240
Number of rows after cleaning: 15240


In [52]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-enero-2020.csv', encoding='latin1', sep=';', index=False)

# datos-abiertos-noviembre_2019.csv 

In [53]:
raw_data = get_input_data(filename = 'datos-abiertos-noviembre_2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15926 entries, 0 to 15925
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   15926 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  15926 non-null  object
 2   CODIGO DE LOCALIDAD                15926 non-null  int64 
 3   LOCALIDAD                          15926 non-null  object
 4   EDAD                               15926 non-null  int64 
 5   UNIDAD                             15926 non-null  object
 6   GENERO                             15926 non-null  object
 7   RED                                15926 non-null  object
 8   TIPO_INCIDENTE                     15926 non-null  object
 9   PRIORIDAD                          15926 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


In [54]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [55]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [56]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy              2192
Engativa              1571
Suba                  1424
 Bosa                 1189
 Ciudad Bolivar       1123
 Puente Aranda         854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
 Santa Fe              613
 Chapinero             577
 Usme                  577
Los Martires           560
 Tunjuelito            551
 Teusaquillo           546
 Barrios Unidos        520
 Antonio Nariño        338
 La Candelaria         102
 Sumapaz                 1
Name: LOCALIDAD, dtype: int64

In [57]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2192
Engativa              1571
Suba                  1424
Bosa                  1189
Ciudad Bolivar        1123
Puente Aranda          854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
Santa Fe               613
Chapinero              577
Usme                   577
Los Martires           560
Tunjuelito             551
Teusaquillo            546
Barrios Unidos         520
Antonio Nariño         338
La Candelaria          102
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [58]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [59]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [60]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)  

SIN_DATO     7011
MASCULINO    4856
FEMENINO     4059
Name: GENERO, dtype: int64

In [61]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    5577
Inconsciente/Paro Cardiorrespiratorio        1587
Dificultad Respiratoria                      1400
Enfermo                                      1066
Convulsiones                                  979
Heridos                                       729
Lesiones personales                           675
Trastorno mental                              601
Intento de suicidio                           600
Dolor torácico                                577
Patología Gineco - obstétrica                 373
Síntomas gastrointestinales                   355
Caída                                         346
Accidente cerebro vascular                    318
Intoxicaciones                                137
SIN_DATO                                      122
Ideas de suicidio                              90
Solicitud Apoyo / Desacato                     83
Incendio estructural                           48
Muerte Natural                                 41


In [62]:
col = 'RED'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Norte      6859
Sur        6000
Privada    3067
Name: RED, dtype: int64

In [63]:
raw_data[col].fillna('SIN_DATO').value_counts()

Norte      6859
Sur        6000
Privada    3067
Name: RED, dtype: int64

In [64]:
col = 'TIPO_INCIDENTE'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [65]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_noviembre_2019.html')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 15886
Number of rows after cleaning: 15886


In [68]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-noviembre-2019.csv', encoding='latin1', sep=';', index=False)

# datos-abiertos-octubre-2019.csv 

In [69]:
raw_data = get_input_data(filename = 'datos-abiertos-octubre-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     18195 non-null  int64 
 1   FECHA_DESPACHO_518   18195 non-null  object
 2   CODIGO DE LOCALIDAD  18195 non-null  int64 
 3   LOCALIDAD            18195 non-null  object
 4   EDAD                 18195 non-null  object
 5   UNIDAD               9503 non-null   object
 6   GENERO               9543 non-null   object
 7   RED                  18195 non-null  object
 8   TIPO_INCIDENTE       18195 non-null  object
 9   PRIORIDAD            18195 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.4+ MB
None


In [70]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        "FECHA_DESPACHO_518"                : "FECHA_INICIO_DESPLAZAMIENTO_MOVIL"
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [71]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [72]:
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   18195 non-null  int64         
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  18195 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   18195 non-null  int64         
 3   LOCALIDAD                          18195 non-null  object        
 4   EDAD                               18195 non-null  object        
 5   UNIDAD                             9503 non-null   object        
 6   GENERO                             9543 non-null   object        
 7   RED                                18195 non-null  object        
 8   TIPO_INCIDENTE                     18195 non-null  object        
 9   PRIORIDAD                          18195 non-null  object        
dtypes: datetime64[ns](1), int64(2), ob

In [73]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2546
 Engativa              1791
 Suba                  1742
 Bosa                  1334
 Ciudad Bolivar        1141
 Puente Aranda          989
 Rafael Uribe Uribe     949
Usaquen                 888
 Fontibon               863
San Cristobal           844
 Chapinero              763
 Los Martires           697
 Santa Fe               689
 Barrios Unidos         680
 Usme                   609
 Tunjuelito             589
 Teusaquillo            536
 Antonio Nariño         419
 La Candelaria          120
 Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [74]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2546
Engativa              1791
Suba                  1742
Bosa                  1334
Ciudad Bolivar        1141
Puente Aranda          989
Rafael Uribe Uribe     949
Usaquen                888
Fontibon               863
San Cristobal          844
Chapinero              763
Los Martires           697
Santa Fe               689
Barrios Unidos         680
Usme                   609
Tunjuelito             589
Teusaquillo            536
Antonio Nariño         419
La Candelaria          120
Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [75]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [76]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [77]:
col = 'PRIORIDAD'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

ALTA        15190
MEDIA        2733
BAJA          228
SIN_DATO       44
Name: PRIORIDAD, dtype: int64

In [78]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


In [79]:
raw_data[col].fillna('SIN_DATO').value_counts()

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


In [80]:
col = 'RED'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [81]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_octubre_2019.html')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [83]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 16559
Number of rows after cleaning: 16559


In [84]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-octubre-2019.csv', encoding='latin1', sep=';', index=False)

# datos-abiertos-septiembre-2019.csv 

In [85]:
raw_data = get_input_data(filename = 'datos-abiertos-septiembre-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17018 entries, 0 to 17017
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   17018 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  17018 non-null  object
 2   CODIGO DE LOCALIDAD                17018 non-null  int64 
 3   LOCALIDAD                          17018 non-null  object
 4   EDAD                               17018 non-null  int64 
 5   UNIDAD                             9371 non-null   object
 6   GENERO                             17018 non-null  object
 7   RED                                17018 non-null  object
 8   TIPO_INCIDENTE                     17018 non-null  object
 9   PRIORIDAD                          17018 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.3+ MB
None


In [86]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [87]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [88]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2398
 Suba                  1634
Engativa               1631
 Bosa                  1294
 Ciudad Bolivar        1138
 Puente Aranda          938
 Rafael Uribe Uribe     854
 Usaquen                829
 San Cristobal          826
Fontibon                809
 Chapinero              749
 Barrios Unidos         616
Tunjuelito              603
Los Martires            593
Usme                    572
 Santa Fe               549
 Teusaquillo            540
 Antonio Nariño         362
 La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [89]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2398
Suba                  1634
Engativa              1631
Bosa                  1294
Ciudad Bolivar        1138
Puente Aranda          938
Rafael Uribe Uribe     854
Usaquen                829
San Cristobal          826
Fontibon               809
Chapinero              749
Barrios Unidos         616
Tunjuelito             603
Los Martires           593
Usme                   572
Santa Fe               549
Teusaquillo            540
Antonio Nariño         362
La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [90]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [91]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [92]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)  

SIN_DATO     7632
MASCULINO    5082
FEMENINO     4304
Name: GENERO, dtype: int64

In [93]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS    6333
INCONSCIENTE/PARO CARDIORRESPIRATORIO        1720
DIFICULTAD RESPIRATORIA                      1386
ENFERMO                                      1137
CONVULSIONES                                 1039
HERIDOS                                       834
INTENTO DE SUICIDIO                           653
DOLOR TORÁCICO                                629
TRASTORNO MENTAL                              621
LESIONES PERSONALES                           593
PATOLOGÍA GINECO - OBSTÉTRICA                 421
SÍNTOMAS GASTROINTESTINALES                   360
ACCIDENTE CEREBRO VASCULAR                    352
CAÍDA                                         334
INTOXICACIONES                                156
IDEAS DE SUICIDIO                             113
VIOLENCIA SEXUAL                               48
INCENDIO ESTRUCTURAL                           40
SIN_DATO                                       32
SANGRADO VAGINAL                               25


In [94]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

2019-09-18 05:09:15    9
2019-09-06 14:47:18    8
2019-09-07 18:29:25    7
2019-09-26 15:06:05    7
2019-09-06 07:53:53    6
                      ..
2019-09-11 10:10:04    1
2019-09-11 10:19:54    1
2019-09-11 10:34:31    1
2019-09-11 10:24:38    1
2019-09-30 23:50:06    1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 15925, dtype: int64

In [95]:
raw_data[col].fillna('SIN_DATO').value_counts()

2019-09-18 05:09:15    9
2019-09-06 14:47:18    8
2019-09-07 18:29:25    7
2019-09-26 15:06:05    7
2019-09-06 07:53:53    6
                      ..
2019-09-11 10:10:04    1
2019-09-11 10:19:54    1
2019-09-11 10:34:31    1
2019-09-11 10:24:38    1
2019-09-30 23:50:06    1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 15925, dtype: int64

In [96]:
col = 'UNIDAD'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,SIN_DATO,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,SIN_DATO,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [97]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_septiembre_2019.html')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [99]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 16267
Number of rows after cleaning: 16267


In [100]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-septiembre-2019.csv', encoding='latin1', sep=';', index=False)

# datos_abiertos_cons_diciembre_2019.csv 

In [101]:
raw_data = get_input_data(filename = 'datos_abiertos_cons_diciembre_2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16243 entries, 0 to 16242
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     16243 non-null  object
 1   FECHA_DESPACHO_518   16243 non-null  object
 2   CODIGO DE LOCALIDAD  16243 non-null  int64 
 3   LOCALIDAD            16243 non-null  object
 4   EDAD                 16243 non-null  int64 
 5   UNIDAD               16243 non-null  object
 6   GENERO               16243 non-null  object
 7   RED                  16243 non-null  object
 8   TIPO_INCIDENTE       16243 non-null  object
 9   PRIORIDAD            16243 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


In [102]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        "FECHA_DESPACHO_518"                : "FECHA_INICIO_DESPLAZAMIENTO_MOVIL"
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


In [103]:
raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(raw_data["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


In [104]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy              2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [105]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [106]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [107]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


In [108]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)  

SIN/DATO     8286
MASCULINO    4405
FEMENINO     3552
Name: GENERO, dtype: int64

In [109]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Heridos accidentales                      7390
Inconsciente / paro cardiorespiratorio    1743
Evento respiratorio                       1404
Convulsión                                1027
Enfermo                                    873
Trastorno mental                           650
Dolor torácico                             611
Intento de Suicidio                        543
Caída de Altura                            377
Patología ginecobstétrica                  371
Accidente Cerebro\nVascular                365
Síntomas gastrointestinales                275
Intoxicación                               153
S/D                                        138
Accidente Tránsito con Heridos/Muertos     108
Amenaza de\nSuicidio                        95
Violencia Sexual                            28
Sangrado Vaginal                            24
Lesiones Personales                         23
Quemaduras                                  16
Electrocución / Rescate                     10
Accidente Cer

In [110]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

2019-12-20 22:05:02    9
2019-12-24 15:02:30    6
2019-12-24 21:08:00    6
2019-12-31 17:39:10    6
2019-12-26 21:34:05    5
                      ..
2019-12-09 14:50:55    1
2019-12-09 15:52:58    1
2019-12-09 15:37:33    1
2019-12-09 16:02:42    1
2019-12-01 07:12:34    1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 15804, dtype: int64

In [111]:
raw_data[col].fillna('SIN_DATO').value_counts()

2019-12-20 22:05:02    9
2019-12-24 15:02:30    6
2019-12-24 21:08:00    6
2019-12-31 17:39:10    6
2019-12-26 21:34:05    5
                      ..
2019-12-09 14:50:55    1
2019-12-09 15:52:58    1
2019-12-09 15:37:33    1
2019-12-09 16:02:42    1
2019-12-01 07:12:34    1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 15804, dtype: int64

In [112]:
col = 'RED'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


In [113]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_diciembre_2019.html')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [115]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 16162
Number of rows after cleaning: 16162


In [116]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-diciembre-2019.csv', encoding='latin1', sep=';', index=False)

In [3]:
filename = 'clean_datos-abiertos-agosto-2019.csv'
agosto   = get_input_data(
bucket   = bucket,
initial_directory = directory,
filename = filename)
agosto["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(agosto["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

print(agosto.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16807 entries, 0 to 16806
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   16807 non-null  int64         
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  16807 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   16807 non-null  int64         
 3   LOCALIDAD                          16807 non-null  object        
 4   EDAD                               16807 non-null  object        
 5   UNIDAD                             16807 non-null  object        
 6   GENERO                             16807 non-null  object        
 7   RED                                16807 non-null  object        
 8   TIPO_INCIDENTE                     16807 non-null  object        
 9   PRIORIDAD                          16807 non-null  object        
 10  MES                               

In [4]:
filename = 'clean_datos-abiertos-diciembre-2019.csv'
diciembre = get_input_data(
bucket   = bucket,
initial_directory = directory,
filename = filename)
diciembre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(diciembre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

print(diciembre.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   16162 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  16162 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   16162 non-null  int64         
 3   LOCALIDAD                          16162 non-null  object        
 4   EDAD                               16162 non-null  int64         
 5   UNIDAD                             16162 non-null  object        
 6   GENERO                             16162 non-null  object        
 7   RED                                16162 non-null  object        
 8   TIPO_INCIDENTE                     16162 non-null  object        
 9   PRIORIDAD                          16162 non-null  object        
dtypes: datetime64[ns](1), int64(2), ob

In [5]:
filename = 'clean_datos-abiertos-julio-2019.csv'
julio = get_input_data(
bucket   = bucket,
initial_directory = directory,
filename = filename)
julio["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(julio["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

print(julio.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13842 entries, 0 to 13841
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   FECHA_INCIDENTE                    13842 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13842 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   13842 non-null  int64         
 3   LOCALIDAD                          13842 non-null  object        
 4   EDAD                               13842 non-null  object        
 5   UNIDAD                             13842 non-null  object        
 6   GENERO                             13842 non-null  object        
 7   RED                                13842 non-null  object        
 8   TIPO_INCIDENTE                     13842 non-null  object        
 9   PRIORIDAD                          13842 non-null  object        
 10  MES                               

In [6]:
filename = 'clean_datos-abiertos-noviembre-2019.csv'
noviembre = get_input_data(
bucket   = bucket,
initial_directory = directory,
filename = filename)
noviembre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(noviembre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

print(noviembre.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   15886 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  15886 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   15886 non-null  int64         
 3   LOCALIDAD                          15886 non-null  object        
 4   EDAD                               15886 non-null  int64         
 5   UNIDAD                             15886 non-null  object        
 6   GENERO                             15886 non-null  object        
 7   RED                                15886 non-null  object        
 8   TIPO_INCIDENTE                     15886 non-null  object        
 9   PRIORIDAD                          15886 non-null  object        
dtypes: datetime64[ns](1), int64(2), ob

In [7]:
filename = 'clean_datos-abiertos-octubre-2019.csv'
octubre = get_input_data(
bucket   = bucket,
initial_directory = directory,
filename = filename)
octubre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(octubre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

print(octubre.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16559 entries, 0 to 16558
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   16559 non-null  int64         
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  16559 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   16559 non-null  int64         
 3   LOCALIDAD                          16559 non-null  object        
 4   EDAD                               16559 non-null  object        
 5   UNIDAD                             9460 non-null   object        
 6   GENERO                             9495 non-null   object        
 7   RED                                16559 non-null  object        
 8   TIPO_INCIDENTE                     16559 non-null  object        
 9   PRIORIDAD                          16559 non-null  object        
dtypes: datetime64[ns](1), int64(2), ob

In [8]:
filename = 'clean_datos-abiertos-septiembre-2019.csv'
septiembre = get_input_data(
bucket   = bucket,
initial_directory = directory,
filename = filename)
septiembre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(septiembre["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

print(septiembre.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16267 entries, 0 to 16266
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   16267 non-null  int64         
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  16267 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   16267 non-null  int64         
 3   LOCALIDAD                          16267 non-null  object        
 4   EDAD                               16267 non-null  int64         
 5   UNIDAD                             16267 non-null  object        
 6   GENERO                             16267 non-null  object        
 7   RED                                16267 non-null  object        
 8   TIPO_INCIDENTE                     16267 non-null  object        
 9   PRIORIDAD                          16267 non-null  object        
dtypes: datetime64[ns](1), int64(3), ob

In [9]:
data = pd.concat([agosto, diciembre,julio,noviembre,octubre,septiembre])
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95523 entries, 0 to 16266
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   81681 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  95523 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   95523 non-null  int64         
 3   LOCALIDAD                          95523 non-null  object        
 4   EDAD                               95523 non-null  object        
 5   UNIDAD                             88424 non-null  object        
 6   GENERO                             88459 non-null  object        
 7   RED                                95523 non-null  object        
 8   TIPO_INCIDENTE                     95523 non-null  object        
 9   PRIORIDAD                          95523 non-null  object        
 10  MES                               

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL,FECHA_INCIDENTE
0,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado,NaN
1,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado,NaN
2,809244194,2019-08-01 00:10:00,13,Teusaquillo,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
3,809253194,2019-08-01 00:12:00,8,Kennedy,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
4,809261194,2019-08-01 00:13:00,11,Suba,71,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,AGOSTO,Traslado,NaN


In [12]:
eliminarAcentos = lambda str: str.translate(str.maketrans("áàäéèëíìïòóöùúüÀÁÄÈÉËÌÍÏÒÓÖÙÚÜ", "aaaeeeiiiooouuuAAAEEEIIIOOOUUU"))

In [13]:
data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL,FECHA_INCIDENTE
0,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado,NaN
1,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado,NaN
2,809244194,2019-08-01 00:10:00,13,Teusaquillo,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
3,809253194,2019-08-01 00:12:00,8,Kennedy,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
4,809261194,2019-08-01 00:13:00,11,Suba,71,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,AGOSTO,Traslado,NaN


In [14]:
profile = ProfileReport(df = data)
profile.to_file(output_file = project_dir + '/reports/'+'profile_raw_data_concat-2019.html')

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
print('Number of rows before cleaning:', data.shape[0] )
data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', data.shape[0] )

Number of rows before cleaning: 95523
Number of rows after cleaning: 95523


In [17]:
data.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-concat.csv', encoding='latin1', sep=';', index=False)